In [3]:
import pandas as pd 
import xarray as xr
import matplotlib.pyplot as plt 
import numpy as np
import os
import seaborn as sns

# import xycmap

import functions


In [12]:
# timestep workaround


folder_path = f'/Volumes/Extreme SSD/202409_paper2_modelruns/30years/SL_{method}/{landcover_id}landcover_{percentile}/{location}_climate_cut'


# read one file to get the time-step
df = functions.calculate_monthly_sediment_yield_all(pd.read_csv(folder_path + '/13a/Sediment.out'), column)
df = df[['month', 'year']]

# elevation langtang
elevation = pd.read_csv(f'/Users/varyabazilova/Desktop/paper2/downscaling_simple/coordinates_and_elevation_with_labels_{location}.csv')[['cellnr2','band_data']] 
elevation = elevation.transpose()

elevation_list = elevation.loc['cellnr2'].tolist()






NameError: name 'landcover_id' is not defined

In [9]:
# Load elevation list and DataFrame once, outside the loop
elevation = pd.read_csv(f'/Users/varyabazilova/Desktop/paper2/downscaling_simple/coordinates_and_elevation_with_labels_{location}.csv')[['cellnr2', 'band_data']]
# elevation_list = elevation.loc['band_data'].tolist()  # Adjust as needed


KeyError: 'band_data'

In [1]:
%%time

# Define the list of landcover indices and percentiles
landcover_indices = [1, 2, 3, 4]
percentiles = ['20percent', '30percent', '40percent', '50percent', '60percent']
location = 'langtang'
method = 'daily' 

# Set the column and other parameters
column = 'Q100'

df = pd.DataFrame()  # Assuming `df` is initialized as needed

# Loop over each landcover index
for landcover_idx in landcover_indices:
    # Loop over each percentile
    for percentile in percentiles:
        print(f"Processing landcover index {landcover_idx} and percentile {percentile}")
        
        # Define folder paths
        folder_path = f'/Volumes/Extreme SSD/202409_paper2_modelruns/30years/SL_{method}/{landcover_idx}landcover_{percentile}/{location}_climate_cut'
        output_folder = f'/Users/varyabazilova/Desktop/paper2/202406_modelruns/30years/2024Nov_output/SL_{method}/output_{percentile}/'
        result_df = pd.DataFrame()

        # Iterate over files in the folder
        for folder_name in os.listdir(folder_path):
            # Skip hidden and .csv files
            if folder_name.endswith('.csv') or folder_name.startswith('.'):
                continue

            file_path = os.path.join(folder_path, folder_name, 'Sediment.out')
            if os.path.isfile(file_path):
                # Read sediment data and calculate monthly sediment yield
                sediments = pd.read_csv(file_path)
                output_df = functions.calculate_monthly_sediment_yield_all(sediments, column)
                
                # Take the specified column and rename it
                name_column = output_df[column]
                column_name = f'{column}_{folder_name}'
                result_df[column_name] = name_column
                result_df.columns = [col[-3:] for col in result_df.columns]

        # Filter and check columns with elevation list
        result_df = result_df[elevation_list]
        if elevation_list == result_df.columns.tolist():
            print("default land cover. monthly data: same")
        else:
            print("not the same")

        # Rename columns according to elevation, merge with timestep, and add land cover info
        result_df.columns = elevation.loc['band_data']
        result_df = pd.concat([df, result_df], axis=1)
        result_df['land_cover'] = f'landcover{landcover_idx}'

        # Save the DataFrame to a CSV file with both landcover and percentile in the filename
        output_filename = f'{location}_monthly_sum_elevation_{column}_landcover{landcover_idx}_{percentile}.csv'
        result_df.to_csv(os.path.join(output_folder, output_filename), index=False)

        print(f"Saved {output_filename}")


NameError: name 'elevation' is not defined